<a href="https://colab.research.google.com/github/amr-elbana/EcoSortAI-Classifier/blob/Preprocessing-amr/Data_Augmentation/Data_Augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Data_Augmentation**



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
from PIL import Image
import numpy as np
from imgaug import augmenters as iaa
import math

# Define the paths
dataset_path = "/path/to/dataset"  # Original dataset path
augmented_path = "/path/to/augmented_dataset"  # Output path for augmented dataset

# Augmentation pipeline
augmenter = iaa.Sequential([
    iaa.Fliplr(0.5),  # Horizontal flip
    iaa.Affine(rotate=(-20, 20)),  # Random rotation
    iaa.Multiply((0.8, 1.2)),  # Brightness adjustment
    iaa.GaussianBlur(sigma=(0, 1.0))  # Apply Gaussian blur
])

# Ensure the output folders exist
for class_name in os.listdir(dataset_path):
    os.makedirs(os.path.join(augmented_path, class_name), exist_ok=True)

# Function to augment images
def augment_images(class_path, output_path, target_count):
    images = [os.path.join(class_path, img) for img in os.listdir(class_path) if img.endswith(('.png', '.jpg', '.jpeg'))]
    existing_count = len(images)
    augment_needed = target_count - existing_count

    if augment_needed <= 0:
        # Copy existing images to the augmented folder
        for img_path in images:
            img_name = os.path.basename(img_path)
            output_img_path = os.path.join(output_path, img_name)
            if not os.path.exists(output_img_path):
                Image.open(img_path).save(output_img_path)
        return

    images = [Image.open(img_path) for img_path in images]
    images_np = [np.array(img) for img in images]

    for i in range(augment_needed):
        img_np = augmenter.augment_image(images_np[i % existing_count])  # Augment image
        augmented_img = Image.fromarray(img_np)
        augmented_img.save(os.path.join(output_path, f"aug_{i+1}.jpg"))

# Get the maximum number of images in any class
max_images = max(len(os.listdir(os.path.join(dataset_path, class_name))) for class_name in os.listdir(dataset_path))

# Calculate target count per class (5x the maximum count)
target_count = max_images * 5

# Augment each class to the target count
for class_name in os.listdir(dataset_path):
    class_path = os.path.join(dataset_path, class_name)
    output_class_path = os.path.join(augmented_path, class_name)
    augment_images(class_path, output_class_path, target_count)


Augmentation complete. Augmented images are saved in: /content/drive/MyDrive/DL_Project/realWaste_Augmented2
